# pyCasandra
By I. Petrov & J. Bareno.

The goal of this document is to port Ivan's FORTRAN code for sputtering calculations to Python. The initil step tis to get it working as a console or notebook package (running in an object). GUI and other functionality will be added later.

## Original FORTRAN CODE:
``` fortran

C	GAS PHASE DENSITY OF SPUTTERED ATOMS
C	DEPOSITION RATE CALCULATIONS FOR PLANAR MAGNETRONS
C
C	DECLARATIONS
	INTEGER NZ
	REAL MA,MG,RG,RA,P,TG,R1,R2,DR,DFI,Z,M,S,ETA,LAMBDA,DRV
	REAL A,B,C,DEP1(30),J(100,100),DEP2(2,30),DEP(30),LALA,KAPA
	REAL QZ,CONC(100,100),DIF
	CHARACTER*2 IO,EL
	CHARACTER*6 FIL,COEFIL
	CHARACTER*1 W,WW
	PARAMETER(PI=3.1415927)
C
C	INTERACTIVE INPUT OF DATA
C
 1	CONTINUE
	TYPE*,'DEPOSITION RATE CALCULATIONS FOR PLANAR MAGNETRONS'
	TYPE*,'ENTER CHEMICAL SYMBOL OF SPUTTERING GAS'
	READ(*,2) IO
	TYPE*,'CHEMICAL SYMBOL OF TARGET ELEMENT'
	READ(*,2) EL
 2	FORMAT(A)
C	RETRIEVAL OF COEFFICIENT FILES
	COEFIL=IO//'.COE'
	OPEN(UNIT=1,FILE=COEFIL,TYPE='OLD')
	READ(1,*) MG,ZG,RG,UG,Q,DEN,KAPA
	CLOSE(UNIT=1)
	KAPA=KAPA*1E-6
C	CALCULATION OF GAS ATOM RADIUS FROM ROBINSON DATA AT 10/2 EV
	RG=SQRT(0.77*0.77 +(0.93*0.93-0.77*0.77)/18*(ZG-18))
	IF (ZG.LT.39) GOTO 4
	RG=SQRT(0.93*0.93+(1.04*1.04-0.93*0.93)/18*(ZG-36))
4	TYPE*,'ION=',IO,MG,ZG,RG,KAPA
	COEFIL=EL//'.COE'
	OPEN(UNIT=1,FILE=COEFIL,TYPE='OLD')
	READ(1,*) MA,ZA,RA,U0,QZ,DEN
	CLOSE(UNIT=1)	
 	OPEN (UNIT=4,NAME='DEPRATE.OUT',TYPE='NEW')
C	CALCULATION OF SPUTTERED ATOM RADIUS BY INTERPOLATION OF ROBINSON'S DATA
C	AT 10/2 eV 
	RA=SQRT(0.77*0.77+(0.93*0.93-0.77*0.77)/18*(ZA-18))
	IF (ZA.LT.39) GOTO 6
	RA=SQRT(0.93*0.93+(1.04*1.04-0.93*0.93)/18*(ZA-36))
6	TYPE*,'TARGET = ',EL,MA,ZA,RA,U0,QZ,DEN
	TYPE*,'DISCHARGE VOLTAGE  IN VOLTS  '
	ACCEPT*,EI
	TYPE*,'DISCHARGE CURRENT IN AMPS'
	ACCEPT*,CUR
	TYPE*,'GAS PRESSURE IN PASCALS'
	ACCEPT*,P
	TYPE*,'GAS TEMPERATURE IN KELVIN'
	ACCEPT*,T0
	TYPE*,'DO YOU WANT TEMPERATURE CORRECTION - (Y)ES?'
	READ(*,2) WW
	TYPE*,'INPUT ACTIVE TARGET AREA IN MILIMETERS'
	TYPE*,'INNER RADIUS OF EROSION ZONE IN mm R1'
	ACCEPT*,R1
	TYPE*,'OUTER RADIUS R2'
	ACCEPT*,R2
	TYPE*,'INPUT N1'
	ACCEPT*,N1
C	TYPE*,'INPUT RADIUS OF CALCULATED VOLUME - ?*R2'
C	ACCEPT*,L5
	NR=3*R2/2
	TYPE*,'dFI=PI/N2   ENTER N2'
	ACCEPT*,N2
	TYPE*,'TARGET SUBSTRATE DISTANCE Z IN mm'
	ACCEPT*,Z
	NZ=Z/2
C
C	CALCULATION OF THE MEAN FREE PATH IN mm
C  
	TG=T0
	M=MG/MA
 3	LAMBDA=1/(2.276516*P/TG*(RG+RA)**2*SQRT(1+MA/MG))
	TYPE*,' LAMBDA = ',LAMBDA
C	
C	CALCULATION OF AVERAGE NUMBER OF COLLISIONS BEFORE THERMALIZATION'
C
	VPER=LOG(SQRT(1+M)+SQRT(M))/4/M**1.5/SQRT(1+M)
	VPER=VPER+(2*M**4+5*M**3+3*M*M-M-1)/4/M/(1+M)**3
	VPER=(1-M)/(1+M)+2*M/(1+M)*VPER
	ETA=LOG(SQRT(8.6174E-5*TG/10/M))
	TYPE*,' LOG(VG/V0)=',ETA,'   V1/V=',VPER
	ETA=ETA/LOG(VPER)
C 	IN THE EXPRESSION FOR ETA WE HAVE FIXED THE AVERAGE ENERGY OF SPUTTERED
C	ATOMS TO BE 10 eV
	TYPE*,' ETA=',ETA,'   M=',M
	LAMBDA=LAMBDA*ETA
	LALA=LAMBDA/TG/10
	TYPE*,'LAMBDA*ETA=',LAMBDA

C
C	CALCULATION OF COLLISIONLESS TRANSPORT  	
C	
	DR=(R2-R1)/N1
	DFI=PI/N2	
	DO 40 R=0,2*R2,10
	S=0
	   	DO 30 K1=0,N1-1,1
	   	RC=R1+(K1+.5)*DR
			DO 20 K2=0,N2-1,1
			FI=(K2+.5)*DFI
			RO=Z*Z+R*R+RC*RC-2*RC*R*COS(FI)
			S=S+RC*EXP(-SQRT(RO)/LAMBDA)/(RO*RO)
 20	CONTINUE
 30	CONTINUE
	DEP1(R/10+1)=Z*Z*DR*DFI*S*2/PI
 	TYPE*,R,DEP1(R/10+1),DEP1(R/10+1)/DEP1(1)
 40	CONTINUE
C	
C	CALCULATION OF THE TOTAL COLISIONLESS DEPOSITION RATE
C
	RTOT=DEP1(1)
	DO 45 I=2,2*R2/10+1,1
	RTOT=RTOT+DEP1(I)*(I-1)*8
 45 	CONTINUE
	RTOT=RTOT*PI*25
	R99=RTOT/PI/(R2**2-R1**2)
	TYPE*,'RDTOT/RSTOT',R99
C
C	SPUTTERING YIELD
C
	IF (MA/MG.GT.1) THEN
	H=.834
	ELSE 
	H=.18
 	END IF
	TH=1.5*U0*(1+1.38*(MG/MA)**H)**2/(4*MA*MG/(MA+MG)**2)
	ALFA=.1+.155*(MA/MG)**.73
	PT=(1+MG/MA)*ZA*ZG*SQRT(ZA**.66666+ZG**.666666)/.0325
	CP=3.56*MG*ZG*ZA*ALFA/(MA+MG)/SQRT(ZA**.666666+ZG**.66666)/U0
	EPS=EI/PT
	SN=3.441*SQRT(EPS)*LOG(EPS+2.718)
	SN=SN/(1+6.355*SQRT(EPS)+EPS*(6.882*SQRT(EPS)-1.708))
	Y=QZ*CP*SN*(1-SQRT(TH/EI))**2
	TYPE*,'E=',EI,'   Y=',Y,' TH=',TH,' ALFA=',ALFA
	TYPE*,'EPS=',EPS,' SN=',SN
C
C	TEMPERATURE CORRECTION
C
	IF (WW.NE.'Y') GOTO 8
	TYPE*,'KAPA=',KAPA,' LALA=',LALA
	AA=CUR*10*Y*(1-R99)/4/PI/KAPA/LALA
	T1=(T0+SQRT(T0**2+4*AA))/2
	IF (ABS(TG-T1).LT.1) GOTO 8
	TG=T1
	TYPE*,'TG=',TG,TG,TG,TG
	GOTO 3
8	TYPE*,'FINAL TEMPERATURE IS ',TG,' K'	
C
C	ENSURING THAT DZ<LAMBDA
C
	DZ=Z/NZ
	IF(DZ.GT.LAMBDA) GOTO 9
	GOTO 10
 9	NZ=Z/LAMBDA
	TYPE*,'NEW VALUE OF NZ--->',NZ
	DZ=Z/NZ
 10	CONTINUE
	TYPE*,'NZ=',NZ
C
C	DIFUSION SOURCE CALCULATION
C
	DRV=3*R2/NR
	DO 80 IR=0,NR-1,1
	R=(IR+.5)*DRV
	  DO 70 IZ=0,NZ-1,1
	  ZC=(IZ+.5)*DZ
	  S=0
	    DO 65 K1=0,N1-1,1
            RC=R1+(K1+.5)*DR
	      DO 60 K2=0,N2-1,1
	      FI=(K2+.5)*DFI
	      RO=SQRT(ZC**2+R**2+RC**2-2*RC*R*COS(FI))
	      S=S+RC*ZC*EXP(-RO/LAMBDA)/LAMBDA/RO**3
 60	CONTINUE	 	
 65	CONTINUE
	J(IR+1,IZ+1)=DR*DFI*S*2/PI
 70 	CONTINUE
 80	CONTINUE
C	
C	CONCENTRATION CALCULATIONS

C
	DO 210 JZ=0,NZ,1
	Z9=JZ*DZ
	DO 200 JR=0,R2,1
	R=JR*2
	SR=0
	 DO 190 IZ=0,NZ-1,1
	 ZC=(IZ+.5)*DZ
	 SZ=0
	  DO 180 IR=0,NR-1,1
	  RC=(IR+.5)*DRV
	   DO 170 K2=0,N2-1,1
  	   FI=(K2+.5)*DFI
	   ROP=SQRT(R*R+RC*RC-2*RC*R*COS(FI))
	   A=(Z9-ZC)/(2*Z)
	   B=(Z9+ZC)/(2*Z)
	   C=ROP/2/Z
	   S0=1./(A*A+C*C)**0.5-1./(B*B+C*C)**0.5
	    DO 160 I=1,1,1
	    S1=1./((I+A)*(I+A)+C*C)**0.5-1./((I-B)*(I-B)+C*C)**0.5
	    S1=S1+1./((I-A)*(I-A)+C*C)**0.5-1./((I+B)*(I+B)+C*C)**0.5
	    S0=S0+S1
 160	    CONTINUE
	   S2=J(IR+1,IZ+1)*DRV*DZ*DFI*RC*S0/8/Z/PI
	   SZ=SZ+S2
 170       CONTINUE
 180      CONTINUE
	 SR=SR+SZ
 190	 CONTINUE
	CONC(JZ+1,JR+1)=2*SR
	TYPE*,'Z=',Z9,'R=',R,'  CONC=',CONC(JZ+1,JR+1)
 200	CONTINUE	
 210    CONTINUE
	
C
C
C	OUTPUT OF RESULTS
C
260	CONTINUE
	TYPE*,'DO YOU WANT THESE RESULTS IN A FILE? (Y)'
	TYPE*,'IF YOU WANT TO END -- INPUT  E, TO START ANEW - RETURN'
	READ(*,2) W
	IF(W.EQ.'E') GOTO 315
	IF(W.NE.'Y') GOTO 1
	WRITE(4,312)'   '
	WRITE(4,304)'MG=',MG,' ZG=',ZG,' RG=',RG,' THERM.COND.=',KAPA
	WRITE(4,304)'MA=',MA,' ZA=',ZA,' RA=',RA,' U0=',U0
	WRITE(4,302)'LAMBDA=',LAMBDA/ETA,' ,10eV,TERM.DIST.=',LAMBDA
 	WRITE(4,302)'ION CURRENT=',CUR,'A VOLTAGE=',EI
	WRITE(4,302)'PRESSURE=',P,'PA Y=',Y
	WRITE(4,304)'TG=',TG,'K R2=',R2,' R1=',R1,' Z=',Z
	WRITE(4,307)'N1=',N1,' N2=',N2,' NR=',NR,' NZ=',NZ
	DO 350 R=0,2*R2,10
	WRITE(4,303)'R=',R,'mm  DSUB=',DEP(R/10+1),' DTAR=',DEP2(2,R/10+1)
	WRITE(4,302)'DIRECT FLUX=',DEP1(R/10+1),' (1)DIFUSED=',DEP2(1,R/10+1)
 350	CONTINUE
	WRITE(4,312)'   '

	DO  299 IR=0,R2,1
	R=IR*2
	DO 298 IZ=0,NZ,1
	ZC=IZ*DZ
	WRITE(4,313)R,ZC,CONC(IZ+1,IR+1)
	TYPE*,CONC(IZ+1,IR+1)
 298	CONTINUE
 299	CONTINUE
 301 	FORMAT(' ',A,F14.7)
 302	FORMAT(' ',A,F14.7,A,F14.7)
 303 	FORMAT(' ',A,F14.7,A,F14.7,A,F14.7)
 304    FORMAT(' ',A,F14.7,A,F14.7,A,F14.7,A,F14.7)	
 307	FORMAT(' ',A,I3,A,I3,A,I3,A,I3)
 311 	FORMAT(' ',A,E14.7)
 312	FORMAT(' ',A)
 313	FORMAT(' 'F14.7,F14.7,F14.7,F14.7)

	GOTO 260
 315	CONTINUE
	CLOSE(UNIT=4)
	STOP
	END
```


## Approach

Define a class containing the data and reproducing the above workflow. The class provides a function to produce the calculated data. Later, auxiliary objects for data input, load, save, and plotting.

### DECLARATIONS and INTERACTIVE INPUT OF DATA
 
This is the section of the code that declares variables and lets user input the data. Start by creating a class to hold the data.

```fortran
C	DECLARATIONS
	INTEGER NZ
	REAL MA,MG,RG,RA,P,TG,R1,R2,DR,DFI,Z,M,S,ETA,LAMBDA,DRV
	REAL A,B,C,DEP1(30),J(100,100),DEP2(2,30),DEP(30),LALA,KAPA
	REAL QZ,CONC(100,100),DIF
	CHARACTER*2 IO,EL
	CHARACTER*6 FIL,COEFIL
	CHARACTER*1 W,WW
	PARAMETER(PI=3.1415927)
C
C	INTERACTIVE INPUT OF DATA
C
 1	CONTINUE
	TYPE*,'DEPOSITION RATE CALCULATIONS FOR PLANAR MAGNETRONS'
	TYPE*,'ENTER CHEMICAL SYMBOL OF SPUTTERING GAS'
	READ(*,2) IO
	TYPE*,'CHEMICAL SYMBOL OF TARGET ELEMENT'
	READ(*,2) EL
 2	FORMAT(A)
C	RETRIEVAL OF COEFFICIENT FILES
	COEFIL=IO//'.COE'
	OPEN(UNIT=1,FILE=COEFIL,TYPE='OLD')
	READ(1,*) MG,ZG,RG,UG,Q,DEN,KAPA
	CLOSE(UNIT=1)
	KAPA=KAPA*1E-6
C	CALCULATION OF GAS ATOM RADIUS FROM ROBINSON DATA AT 10/2 EV
	RG=SQRT(0.77*0.77 +(0.93*0.93-0.77*0.77)/18*(ZG-18))
	IF (ZG.LT.39) GOTO 4
	RG=SQRT(0.93*0.93+(1.04*1.04-0.93*0.93)/18*(ZG-36))
4	TYPE*,'ION=',IO,MG,ZG,RG,KAPA
	COEFIL=EL//'.COE'
	OPEN(UNIT=1,FILE=COEFIL,TYPE='OLD')
	READ(1,*) MA,ZA,RA,U0,QZ,DEN
	CLOSE(UNIT=1)	
    ```


In [7]:
import numpy as np

class SpData():
    '''Class to contain the parameters needed for sputtering calculations.
    Actual calculation and data verification are handled by derived classes.'''
    
    def __init__():
        '''SpData constructor. Initialize data to FALSE, zero, empty, etc.
        Used to document variables, FORTRAN type definition and meaning'''
        
        NZ = 0 # INTEGER
        MA = 0. # REAL
        MG = 0. # REAL
        RG = 0. # REAL
        RA = 0. # REAL
        P = 0. # REAL
        TG = 0. # REAL
        R1 = 0. # REAL
        R2 = 0. # REAL
        DR = 0. # REAL
        DFI = 0. # REAL
        Z = 0. # REAL
        M = 0. # REAL
        S = 0. # REAL
        ETA = 0. # REAL
        LAMBDA = 0. # REAL
        DRV = 0. # REAL
        
        A = 0. # REAL
        B = 0. # REAL
        C = 0. # REAL
        DEP1 = np.zeros(30) # REAL(30)
        J = np.zeros(100,100) # REAL(100,100)
        DEP2 = np.zeros(2,30) # REAL(2,30)
        DEP =np.zeros(30) #REAL(30)
        LALA = 0. # REAL
        KAPA = 0. # REAL
        
        QZ = 0. # REAL
        CONC = np.zeros(100,100) # REAL(100,100)
        DIF = 0. # REAL
        
        IO = "" # CHARACTER*2 Chemical symbol of sputter gas. 
        # Used to load atomic data from file
        EL = "" # CHARACTER*2 Chemical symbol of target element
        # Used to load atomic data from file
        
        FIL = "" # CHARACTER*6
        COEFIL  = "" # CHARACTER*2
        W = "" # CHARACTER*1
        WW = "" # CHARACTER*1
        #PARAMETER(PI=3.1415927)

### Atomic parameters
Atomic parameters originally stored and loaded from .COE files. Here, I scrub these files to put all of them together into a single data file or pickled object.

All files come with 7 parameters 5 in first line and 2 in second line.

In [17]:
from os import listdir
COE_files = [] 
for f in listdir('.'):
    if f[-3:]=='COE':
        COE_files.append(f)
    
print(COE_files)

['SI.COE', 'PD.COE', 'IR.COE', 'PT.COE', 'SN.COE', 'PB.COE', 'RH.COE', 'U.COE', 'OS.COE', 'ZR.COE', 'C.COE', 'AL.COE', 'V.COE', 'CD.COE', 'MO.COE', 'NB.COE', 'MN.COE', 'W.COE', 'CR.COE', 'BE.COE', 'TI.COE', 'CU.COE', 'NI.COE', 'XE.COE', 'AG.COE', 'CO.COE', 'AR.COE', 'ZN.COE', 'TA.COE', 'AU.COE', 'GE.COE', 'RE.COE', 'KR.COE', 'SC.COE', 'RU.COE', 'FE.COE']


In [26]:
import pandas as pd
COE_dir ={}
COE_df ={}
for fname in COE_files:
    with open(fname, 'r') as f:
        fdat = f.readlines()
    el = fname.split('.')[0]
    # Read parameters:
    el_dir={}
    fl =fdat[0].split()
    el_dir['M']=float(fl[0])
    el_dir['Z']=float(fl[1])
    el_dir['R']=float(fl[2])
    el_dir['U']=float(fl[3])
    el_dir['Q']=float(fl[4])
    fl =fdat[1].split()
    el_dir['DEN']=float(fl[0])
    el_dir['KAPA']=float(fl[1])
    
    el_ser = pd.Series(el_dir)
    COE_df[el]=el_ser
    COE_dir[el] = el_dir


In [27]:
COE_df=pd.DataFrame(COE_df)

In [28]:
COE_df

,SI,PD,IR,PT,SN,PB,RH,U,OS,ZR,...,AR,ZN,TA,AU,GE,RE,KR,SC,RU,FE
M,28.00,106.40,192.20,195.00,118.69,207.19,103.00,238.00,190.20,91.22,...,39.9480,65.37,180.948,196.97,72.59,186.20,83.8000,45.0,101.00,55.85
Z,14.00,46.00,77.00,78.00,50.00,82.00,45.00,92.00,76.00,40.00,...,18.0000,30.00,73.000,79.00,32.00,75.00,36.0000,21.0,44.00,26.00
R,2.00,1.00,6.90,3.00,1.00,2.00,5.00,2.00,1.00,6.25,...,0.9800,2.00,1.490,1.00,2.00,1.00,1.1200,1.6,2.00,2.00
U,4.63,3.89,6.94,5.84,3.14,2.03,5.75,5.55,8.17,6.25,...,5.0000,1.35,8.100,3.81,3.85,8.03,0.0000,4.9,6.74,4.28
Q,0.75,1.00,1.14,0.94,0.26,1.00,1.13,0.68,1.13,0.66,...,1.0000,1.00,0.750,0.94,0.70,1.07,0.0000,1.0,1.32,1.07
DEN,2.33,1.00,6.00,9.00,1.00,2.00,1.00,2.00,9.00,9.00,...,5.0000,1.00,16.600,9.00,2.00,9.00,0.0000,3.0,2.00,9.00
KAPA,1.00,1.00,9.00,9.00,1.00,9.00,1.00,2.00,0.00,9.00,...,0.0002,1.00,9.000,9.00,2.00,9.00,0.0001,777.0,2.00,90.00


In [35]:
COE_dir.keys()

dict_keys(['SI', 'PD', 'IR', 'PT', 'SN', 'PB', 'RH', 'U', 'OS', 'ZR', 'C', 'AL', 'V', 'CD', 'MO', 'NB', 'MN', 'W', 'CR', 'BE', 'TI', 'CU', 'NI', 'XE', 'AG', 'CO', 'AR', 'ZN', 'TA', 'AU', 'GE', 'RE', 'KR', 'SC', 'RU', 'FE'])

In [34]:
COE_df[['AR','KR','XE']]

,AR,KR,XE
M,39.9480,83.8000,131.30000
Z,18.0000,36.0000,54.00000
R,0.9800,1.1200,1.31000
U,5.0000,0.0000,0.00000
Q,1.0000,0.0000,0.00000
DEN,5.0000,0.0000,0.00000
KAPA,0.0002,0.0001,0.00003


Create a class to store atom coefficients. Then saved as pickled object for distribution. Advanced users can use this class to modify atom parameters.


In [76]:
class AtomCOE():
    '''Class to store and retrieve atom parameters.
    COEffiecients stored in two dictionaries of parameter vectors, 
    one for gases and one for target elements.
    index dict gives the index in the vector for property of interest
    '''
    
    def __init__(self):
        self.gas={}
        self.target={}
        self.index ={
            'M':0, 'Z':1,'R':2, 'U' :3,
            'Q': 4, 'DEN': 5, 'KAPA':6
        }
        
    def _gas_str(self):
        '''Return string of gas'''
    
    def __str__(self):
        ret_str ="Atomic coeficients for CasandraSputtering Calculations \n\n"
        ret_str += "Gases:\n"
        ret_str += "El\tMG\tZG\tRG\tUG\tQ\tDEN\tKAPA\n"
        for key, val in self.gas.items():
            ret_str += key
            for x in val:
                ret_str += '\t' + str(x)
            ret_str += '\n' 
        ret_str += "\n Target materials:\n"
        ret_str += "El\tMA\tZA\tRA\tUA\tQ\tDEN\tKAPA\n"
        for key, val in self.target.items():
            ret_str += key
            for x in val:
                ret_str += '\t' + str(x)
            ret_str += '\n'
        return(ret_str)
    
    def get_G(self, el, prop):
        ind = self.index[prop]
        return(self.gas[el][ind])
    
    def get_A(self, el, prop):
        ind = self.index[prop]
        return(self.target[el][ind])
    

In [77]:
atCOE = AtomCOE()

for fname in COE_files:
    with open(fname, 'r') as f:
        fdat = f.readlines()
    el = fname.split('.')[0]
    if len(el)==2:
        el = list(el)
        el[-1]=el[-1].lower()
        el = "".join(el)
    
    # Read parameters:
    el_vec=[]
    el_vec.extend([float(x) for x in fdat[0].split()])
    el_vec.extend([float(x) for x in fdat[1].split()])
    if el in ['Ar', 'Kr', 'Xe']:
        atCOE.gas[el]= el_vec
    else:
        atCOE.target[el]=el_vec


In [78]:
print(atCOE)

Atomic coeficients for CasandraSputtering Calculations 

Gases:
El	MG	ZG	RG	UG	Q	DEN	KAPA
Xe	131.3	54.0	1.31	0.0	0.0	0.0	3e-05
Ar	39.948	18.0	0.98	5.0	1.0	5.0	0.0002
Kr	83.8	36.0	1.12	0.0	0.0	0.0	0.0001

 Target materials:
El	MA	ZA	RA	UA	Q	DEN	KAPA
Si	28.0	14.0	2.0	4.63	0.75	2.33	1.0
Pd	106.4	46.0	1.0	3.89	1.0	1.0	1.0
Ir	192.2	77.0	6.9	6.94	1.14	6.0	9.0
Pt	195.0	78.0	3.0	5.84	0.94	9.0	9.0
Sn	118.69	50.0	1.0	3.14	0.26	1.0	1.0
Pb	207.19	82.0	2.0	2.03	1.0	2.0	9.0
Rh	103.0	45.0	5.0	5.75	1.13	1.0	1.0
U	238.0	92.0	2.0	5.55	0.68	2.0	2.0
Os	190.2	76.0	1.0	8.17	1.13	9.0	0.0
Zr	91.22	40.0	6.25	6.25	0.66	9.0	9.0
C	12.0	6.0	0.77	7.31	2.5	2.26	9.0
Al	26.98	13.0	1.43	3.39	1.03	2.7	0.0
V	51.0	23.0	1.0	5.31	0.835	6.1	2.0
Cd	112.4	48.0	1.0	1.16	1.0	1.0	1.0
Mo	95.94	42.0	1.4	6.82	0.86	10.22	2.0
Nb	92.9	41.0	1.0	7.57	0.905	2.0	2.0
Mn	55.0	25.0	1.0	2.92	1.09	8.0	9.0
W	183.85	74.0	1.41	8.9	0.9	19.3	9.0
Cr	52.0	24.0	2.0	4.1	1.22	9.0	0.0
Be	9.012	4.0	1.0	3.32	2.0	8.0	9.0
Ti	47.9	22.0	1.47	4.85	0.54	4.51	0.0


In [82]:
import pickle
with open('atoms.coe', 'wb') as f:
    pickle.dump(atCOE, f)